# GGA 能量相关基础

这一节我们简要了解如何导出 GGA 能量．这里所指的 GGA 允许杂化，但不允许 LDA 或 meta-GGA；当泛函核的参数量更改时，尽管程序大体构成不会更变，但细节的代码变动会非常大．而从实现的角度上讲，杂化泛函与普通泛函的区别仅仅是交换积分的系数差别，代码变动还不大．因此，这一节我们暂时专注于 GGA．

由于目前的 GGA 种类繁多，我们在这里就以 B3LYP 举例．

In [ ]:
import numpy as np
import scipy
from pyscf import scf, gto, lib, dft
from functools import partial

import sys
sys.path.append('../../src')
from utilities import val_from_fchk

np.einsum = partial(np.einsum, optimize=["greedy", 1024 ** 3 * 2 / 8])
np.set_printoptions(5, linewidth=150, suppress=True)

In [ ]:
mol = gto.Mole()
mol.atom = """
O  0.0  0.0  0.0
O  0.0  0.0  1.5
H  1.5  0.0  0.0
H  0.0  0.7  1.5
"""
mol.basis = "6-31G"
mol.verbose = 0
mol.build()

## PySCF GGA 计算

PySCF 的 GGA 计算在代码上与 HF 计算稍有不同；除了自洽场的类不同之外，我们还需要定义格点与泛函形式．

### 格点定义

下面是我们以后会经常使用的格点；这部分的格点配置应当与 Gaussian 的选项 `Integral(Grid=99590)` 非常接近，但一定仍然有所差别．

In [ ]:
grids = dft.gen_grid.Grids(mol)
grids.atom_grid = (99, 590)
grids.becke_scheme = dft.gen_grid.stratmann
grids.prune = None
grids.build()

### 自洽场类定义

涉及 GGA 的自洽场以后一般就使用 B3LYP．B3LYP 仍然包含一部分 VWN 的贡献；PySCF 默认的 B3LYP 为 VWN5，而 Gaussian 默认则为 VWN3．我们采用 Gaussian 的默认形式，因此泛函名为 `b3lypg`．

In [ ]:
scf_eng = dft.RKS(mol)
scf_eng.grids = grids
scf_eng.xc = "b3lypg"
scf_eng.conv_tol = 1e-12
scf_eng.conv_tol_grad = 1e-10
scf_eng.kernel()

尽管 GGA 与 RHF 使用不同的类进行自洽计算：

In [ ]:
print(dft.rks.RKS)
print(scf.rhf.RHF)
print(dft.rks.RKS.__base__)

但 GGA 的基类是 RHF，因此绝大多数 RHF 可以计算的量，原则上 GGA 也可以同样地计算，只是意义有可能会不同；我们在文档最后会考虑这些不同之处．下面的两段隐藏代码，与 RHF 一样地，生成自洽过程前后所产生的各种变量与张量．

In [ ]:
nmo = nao = mol.nao
natm = mol.natm
nocc = mol.nelec[0]
nvir = nmo - nocc
so = slice(0, nocc)
sv = slice(nocc, nmo)
sa = slice(0, nmo)

In [ ]:
C       = scf_eng.mo_coeff
e       = scf_eng.mo_energy
Co      = C[:, so]
Cv      = C[:, sv]
eo      = e[so]
ev      = e[sv]
D       = 2 * Co @ Co.T
F_0_ao  = scf_eng.get_fock()
F_0_mo  = C @ F_0_ao @ C.T
H_0_ao  = scf_eng.get_hcore()
H_0_mo  = C @ H_0_ao @ C.T
eri0_ao = mol.intor("int2e")
eri0_mo = np.einsum("uvkl, up, vq, kr, ls -> pqrs", eri0_ao, C, C, C, C)
mo_occ  = scf_eng.mo_occ

最后我们将计算结果与 [Gaussian 结果](include/basic_gga/b3lyp_energy.gjf) 进行比对

In [ ]:
np.allclose(
    val_from_fchk("SCF Energy", "include/basic_gga/b3lyp_energy.fchk"),
    scf_eng.e_tot
)

## 格点与泛函相关定义

我们单列一小节对格点与泛函相关的内容作定义．这部分的代码量很大，为了避免以后零散的说明、以及大段重复代码出现在笔记中，我们统一在这一节进行记号的定义与代码的书写，即使我们在这一节中不打算用上所有的记号定义．

格点积分有关的量包括格点本身的性质、轨道或密度及其梯度格点，以及泛函核格点．轨道或密度在原子坐标下的梯度我们会在将来叙述，这里的梯度指的是电子坐标的梯度．

<div class="alert alert-info">

**记号说明**

尽管程序中需要将格点显式地写出，但出于简化公式表达的目的，我们就不再在公式表达式中写出格点角标；而与格点直接关联的权重也会归并到泛函核中．

* $\rho, \gamma$ 分别代表密度与 $\rho_r \rho_r$

* $f$ 代表泛函核；泛函核满足关系 $E_\mathrm{xc} = \int f(\rho(\boldsymbol{r})) \rho(\boldsymbol{r}) \, \mathrm{d} \boldsymbol{r} = f \rho$

* $f_\rho = \partial_\rho (f \rho)$；**注意**不是 $\partial_\rho f$

* $f_\gamma = \partial_\gamma (f \rho)$

* $f_{\rho \gamma} = \partial_\rho \partial_\gamma (f \rho)$，其它高阶导数同理

* $\rho_r = \partial_r \rho$

* $\rho_{rw} = \partial_r \partial_w \rho$

* $c_\mathrm{x}$ 代表杂化泛函中的精确交换积分贡献．

</div>

### 泛函核无关部分

<div class="alert alert-warning">

**注意**

这里使用了 `pyscf.dft.numint.NumInt.block_loop` 函数；但我们应当注意到该函数原则上并不返回全部的轨道格点信息，而是返回一个迭代器．使用迭代器的目的是分散计算量．如果大体系或内存空间不够时，下面的代码会失效．

</div>

* `ni` 与格点积分有关的 PySCF 类

* `ngrid` 格点数量

* `grid_ao` 各阶电子坐标偏导数的 AO 轨道格点

* `grid_weight` 格点权重

* `grid_ao_0` 轨道格点 $\phi_\mu$

* `grid_ao_1` 轨道格点一阶导数 $\phi_{r \mu} = \partial_r \phi_\mu$

* `grid_ao_2` 轨道格点二阶导数 $\phi_{r w \mu} = \partial_r \partial_w \phi_\mu$

* `grid_ao_2T` 轨道格点二阶导数，但两个坐标分量打包在一个维度中 $\phi_{T \mu} = \partial_{T_1} \partial_{T_2} \phi_\mu$

* `grid_ao_3T` 轨道格点三阶导数，但其中两个坐标分量打包在一个维度中 $\phi_{T r \mu} = \partial_{T_1} \partial_{T_2} \partial_r \phi_\mu$

* `grid_rho_0` 密度格点 $\rho = D_{\mu \nu} \phi_\mu \phi_\nu$

* `grid_rho_1` 密度格点一阶导数 $\rho_r = \partial_r \rho = 2 D_{\mu \nu} \phi_{r \mu} \phi_\nu$

* `grid_rho_2` 密度格点二阶导数 $\rho_{rw} = \partial_r \partial_w \rho = 2 D_{\mu \nu} (\phi_{r w \mu} \phi_\nu + \phi_{r \mu} \phi_{w \nu})$

* `grid_rho_01` 密度格点与其一阶导数的合并张量；只用于生成泛函核导数

In [ ]:
# PySCF Numerical Integral Engine

ni = dft.numint.NumInt()

# Grid Information

ngrid = grids.weights.size
grid_weight = grids.weights

grid_ao = np.empty((20, ngrid, nao))  # 20 at first dimension is related to 3rd derivative of orbital

current_grid_count = 0
for ao, _, _, _ in ni.block_loop(mol, grids, nao, 3, 2000):
    grid_ao[:, current_grid_count:current_grid_count+ao.shape[1]] = ao
    current_grid_count += ao.shape[1]

# AO Orbital Grid

grid_ao_0  = grid_ao[0]
grid_ao_1  = grid_ao[1:4]
grid_ao_2T = grid_ao[4:10]

XX, XY, XZ, YY, YZ, ZZ = range(4, 10)
XXX, XXY, XXZ, XYY, XYZ, XZZ, YYY, YYZ, YZZ, ZZZ = range(10, 20)

grid_ao_2 = np.array([
    [grid_ao[XX], grid_ao[XY], grid_ao[XZ]],
    [grid_ao[XY], grid_ao[YY], grid_ao[YZ]],
    [grid_ao[XZ], grid_ao[YZ], grid_ao[ZZ]],
])
grid_ao_3T = np.array([
    [grid_ao[XXX], grid_ao[XXY], grid_ao[XXZ], grid_ao[XYY], grid_ao[XYZ], grid_ao[XZZ]],
    [grid_ao[XXY], grid_ao[XYY], grid_ao[XYZ], grid_ao[YYY], grid_ao[YYZ], grid_ao[YZZ]],
    [grid_ao[XXZ], grid_ao[XYZ], grid_ao[XZZ], grid_ao[YYZ], grid_ao[YZZ], grid_ao[ZZZ]],
])

# Density Grid

grid_rho_01 = np.einsum("uv, rgu, gv -> rg", D, grid_ao[0:4], grid_ao_0)
grid_rho_01[1:] *= 2
grid_rho_0 = grid_rho_01[0]
grid_rho_1 = grid_rho_01[1:4]
grid_rho_2 = (
    + 2 * np.einsum("uv, rgu, wgv -> rwuv", D, grid_ao_1, grid_ao_1)
    + 2 * np.einsum("uv, rwgu, gv -> rwuv", D, grid_ao_2, grid_ao_0)
)

<div class="alert alert-info">

**任务**

1. 相比于以前我公式的推导，这里公式的记号因为去除了格点记号，因此不能单纯地从角标查看张量维度的信息了．请自行查看上述各个张量的维度信息．对于张量维度的把握是正确处理量化公式与 numpy 程序实现的第一步．

2. `grid_ao_2T` 与 `grid_ao_3T` 是利用张量对称性而产生的变量；是为了在将来的计算中减小计算量，牺牲变量名的直观意义与代码可读性而提高效率的做法．请指出 `grid_ao_2T` 与 `grid_ao_2` 之间的关系，并尝试从 `grid_ao_2` 构建 `grid_ao_2T`．

3. 生成密度格点的另一个看起来更合理的做法是 $\rho_r = D_{\mu \nu} (\phi_{r \mu} \phi_\nu + \phi_\mu \phi_{r \nu})$．试问为何 $\rho_r = 2 D_{\mu \nu} \phi_{r \mu} \phi_\nu$ 也是正确的？

   这是一个非常关键的问题，因为很多时候我们需要利用 $\mu, \nu$ 的对称性，但不是所有 $\mu, \nu$ 都具有对称性．对这类问题的理解会极大帮助我们正确地推导公式并作公式与代码的简化．
   
4. (可选) 上面我们大量地从 `grid_ao` 与 `grid_rho_01` 中截取张量．请指出哪些截取过程近乎没有代价 (包括耗时、耗内存)．

</div>

### 泛函核相关部分

* `c_x` 泛函杂化系数 $c_\mathrm{x}$

* `grid_exc` 带权重的 $f$

* `grid_fr` 带权重的 $f_\rho$

* `grid_fg` 带权重的 $f_\gamma$

* `grid_frr` 带权重的 $f_{\rho \rho}$

* `grid_frg` 带权重的 $f_{\rho \gamma}$

* `grid_fgg` 带权重的 $f_{\gamma \gamma}$

In [ ]:
cx = ni.hybrid_coeff(scf_eng.xc)
grid_exc, grid_vxc, grid_fxc = ni.eval_xc(scf_eng.xc, grid_rho_01, deriv=2)[:3]
grid_fr, grid_fg = grid_vxc[0:2]
grid_frr, grid_frg, grid_fgg = grid_fxc[0:3]
grid_exc *= grid_weight
grid_fr  *= grid_weight
grid_fg  *= grid_weight
grid_frr *= grid_weight
grid_frg *= grid_weight
grid_fgg *= grid_weight

## 实现参考

这一份笔记的实现参考的前三个是 GGA 的 SCF 过程中所必须得到与确认的项；而从第四个开始，则是一些性质的验证．

前三个问题是电子数、交换相关能与交换相关势．这些可以通过函数 `pyscf.dft.numint.nr_rks` 生成．

In [ ]:
ni.nr_rks.__func__

In [ ]:
xc_n, xc_e, xc_v = ni.nr_rks(mol, grids, scf_eng.xc, D)

### 电子数

前面已经指出，为了简化公式，我们通常不在公式中写出格点角标与权重本身；但在程序实现上，我们始终要记得每个格点在被求和时都要乘以其系数．以后的代码中，格点的权重已经被打包到泛函核中；但在类如电子数的简单积分等问题里，泛函核值的格点是不需要的，因此我们有必要把格点权重放在代码中．

In [ ]:
np.allclose(
    (grid_weight * grid_rho_0).sum(),
    xc_n
)

In [ ]:
xc_n

### 交换相关能 $E_\mathrm{xc}$

$E_\mathrm{xc} = f \rho$

In [ ]:
np.allclose(
    (grid_exc * grid_rho_0).sum(),
    xc_e
)

<div class="alert alert-info">

**任务**

1. 我们只说生成了密度格点，但我们其实还没有进行过量纲分析．请根据电子数格点积分的程序结果，判断 `grid_rho_0` 作为体系密度 $\rho$，其每个元素的单位是什么．

   提示：格点权重认为是无量纲值．

2. 根据上一问的结果与交换相关能的生成代码，请判断 `grid_exc` 作为泛函核 $f$，其每个元素的单位．

</div>

### 交换相关势 $v_{\mu \nu}^\mathrm{xc} [\rho]$

$v_{\mu \nu}^\mathrm{xc} [\rho] = f_\rho \phi_\mu \phi_\nu + 2 f_\gamma \rho_r (\phi_{r \mu} \phi_{\nu} + \phi_{\mu} \phi_{r \nu}) $

In [ ]:
np.allclose(
    + np.einsum("g, gu, gv -> uv", grid_fr, grid_ao_0, grid_ao_0)
    + 2 * np.einsum("g, rg, rgu, gv -> uv", grid_fg, grid_rho_1, grid_ao_1, grid_ao_0)
    + 2 * np.einsum("g, rg, gu, rgv -> uv", grid_fg, grid_rho_1, grid_ao_0, grid_ao_1),
    xc_v
)

<div class="alert alert-info">

**任务**

1. 以前我们生成密度的一阶梯度 $\rho_r$ 时提到，那时的公式与代码中的 $\phi_{r \mu} \phi_\nu + \phi_\mu \phi_{r \nu}$ 可以简化为 $2 \phi_{r \mu} \phi_\nu$ 进行计算．试问现在生成交换相关势时，是否也可以这么简化？为什么？

2. (可选) 你可能已经理解不可以像生成 $\rho_r$ 时那样简化 $\phi_{r \mu} \phi_\nu + \phi_\mu \phi_{r \nu}$ 了，但你仍然可以依靠 $\mu, \nu$ 角标的对称性质，对上面代码块的计算耗时优化到原先的 2/3 倍．请提出你的解决方案．试问代码效率是否还能进一步优化？

3. (可选) 我们没有给出交换相关势的推导过程．请尝试推导交换相关势，并将你的推导与上面的公式对应，以熟悉这份笔记的记号体系．

   并且指出，既然 $v_{\mu \nu}^\mathrm{xc} [\rho]$ 是与密度有关的量，那么其构成中，哪些张量会具体地因与体系密度不同而变化，而那些则始终不变？你是否认为用 $D_{\kappa \lambda}$ 替换掉方括号中的 $\rho$ 是合理的行为？

</div>

### Fock 矩阵 (GGA) $F_{\mu \nu} [X_{\kappa \lambda}]$

$F_{\mu \nu}[X_{\kappa \lambda}] = h_{\mu \nu} + J_{\mu \nu}[X_{\kappa \lambda}] - \frac{1}{2} c_\mathrm{x} K_{\mu \nu}[X_{\kappa \lambda}] + v_{\mu \nu}^\mathrm{xc} [X_{\kappa \lambda}]$

如果 $X_{\kappa \lambda}$ 是密度矩阵 $D_{\mu \nu}$，那么程序是很容易给出的：

In [ ]:
np.allclose(
    scf_eng.get_fock(dm=D),
    scf_eng.get_hcore() + scf_eng.get_j(dm=D) - 0.5 * cx * scf_eng.get_k(dm=D)
    + np.einsum("g, gu, gv -> uv", grid_fr, grid_ao_0, grid_ao_0)
    + 2 * np.einsum("g, rg, rgu, gv -> uv", grid_fg, grid_rho_1, grid_ao_1, grid_ao_0)
    + 2 * np.einsum("g, rg, gu, rgv -> uv", grid_fg, grid_rho_1, grid_ao_0, grid_ao_1)
)

但是如果 $X_{\kappa \lambda}$ 现在是一个任意矩阵，那么我们至少要多做三件事：生成关于 $X_{\kappa \lambda}$ 的广义密度 $\rho^X$；重新构建以 $\rho^X$ 为基础的泛函核偏导数 $f_{\rho^X}, f_{\gamma^X}$；随后代回交换相关势的公式中．

以后的代码中，我们一般不会遇到需要生成关于广义密度的泛函核导数的计算中；因此下面的代码仅作展示用途．

我们仍然在实际程序计算中假定 $X_{\kappa \lambda}$ 是对称矩阵．

In [ ]:
X = np.random.random((nao, nao))
X += X.T

grid_X_01 = np.einsum("uv, rgu, gv -> rg", X, grid_ao[0:4], grid_ao_0)
grid_X_01[1:] *= 2
grid_X_0 = grid_X_01[0]
grid_X_1 = grid_X_01[1:4]
grid_excX, (grid_frX, grid_fgX, _, _) = ni.eval_xc(scf_eng.xc, grid_X_01, deriv=1)[:2][:2]
grid_excX *= grid_weight
grid_frX *= grid_weight
grid_fgX *= grid_weight

np.allclose(
    scf_eng.get_fock(dm=X),
    scf_eng.get_hcore() + scf_eng.get_j(dm=X) - 0.5 * cx * scf_eng.get_k(dm=X)
    + np.einsum("g, gu, gv -> uv", grid_frX, grid_ao_0, grid_ao_0)
    + 2 * np.einsum("g, rg, rgu, gv -> uv", grid_fgX, grid_X_1, grid_ao_1, grid_ao_0)
    + 2 * np.einsum("g, rg, gu, rgv -> uv", grid_fgX, grid_X_1, grid_ao_0, grid_ao_1)
)

### 体系总能量 (GGA) $E_\mathrm{elec}[X_{\mu \nu}]$

$E_\mathrm{elec}[X_{\mu \nu}] = (h_{\mu \nu} + \frac{1}{2} J_{\mu \nu} [X_{\kappa \lambda}] - \frac{1}{4} c_\mathrm{x} [X_{\kappa \lambda}])  X_{\mu \nu} + f \rho^X$

与 RHF 的情况相同，`pyscf.dft.rks.energy_elec` 会返回两个数值，其中的第一个是体系电子态总能量，第二个则是去除 Hamiltonian Core 的能量．

我们接着刚刚使用的 $X{\mu \nu}$ 来生成体系的电子态总能量：

In [ ]:
np.allclose(
    scf_eng.energy_elec(dm=X)[0],
    ((scf_eng.get_hcore() + 0.5 * scf_eng.get_j(dm=X) - 0.25 * cx * scf_eng.get_k(dm=X)) * X).sum()
    + (grid_excX * grid_X_0).sum()
)

以及去除 Hamiltonian Core 的能量：

In [ ]:
np.allclose(
    scf_eng.energy_elec(dm=X)[1],
    ((0.5 * scf_eng.get_j(dm=X) - 0.25 * cx * scf_eng.get_k(dm=X)) * X).sum()
    + (grid_excX * grid_X_0).sum()
)

### 重叠积分 $S_{\mu \nu}$

$S_{\mu \nu} = \langle \mu | \nu \rangle = \phi_\mu \phi_\nu$

In [ ]:
np.allclose(
    mol.intor("int1e_ovlp"),
    np.einsum("g, gu, gv -> uv", grid_weight, grid_ao_0, grid_ao_0),
    atol=1e-7
)

### 动能积分 $T_{\mu \nu}$

$T_{\mu \nu} = \langle \mu | -\frac{1}{2} \partial_r^2 | \nu \rangle = -\frac{1}{2} \phi_{\mu} \phi_{rr \nu} $

In [ ]:
np.allclose(
    mol.intor("int1e_kin"),
    - 0.5 * np.einsum("g, gu, gvr -> uv", grid_weight, grid_ao_0, grid_ao_2.diagonal(axis1=0, axis2=1)),
    atol=1e-6
)

## XYG3 能量计算

刚才我们已经了解了如何使用自定义的密度来计算体系总能量；而上一份笔记中我们回顾了 MP2 能量的计算；那么也就意味着实现 XYG3 也并非困难．我们在这里使用 PySCF 的高层函数做这件事．

以后，我们的默认的非自洽 GGA 泛函为 XYG3 泛函 (除去 MP2 部分)．下面代码块的 `nc_eng` 是非自洽 (Non-Consistent) 泛函能量的类，而 `cc_nc` 表示 PT2 的掺杂比例 $c_\mathrm{c}^\mathrm{nc}$．尽管我们未必会用到所有下述生成的非自洽泛函核格点，但以后的工作中，我们可以使用它们．

In [ ]:
nc_eng = dft.RKS(mol)
nc_eng.xc = "0.8033*HF - 0.0140*LDA + 0.2107*B88, 0.6789*LYP"
nc_eng.grids = grids
cc_nc = 0.3211

cx_nc = ni.hybrid_coeff(nc_eng.xc)
grid_exc_nc, grid_vxc_nc, grid_fxc_nc = ni.eval_xc(nc_eng.xc, grid_rho_01, deriv=2)[:3]
grid_fr_nc, grid_fg_nc = grid_vxc_nc[0:2]
grid_frr_nc, grid_frg_nc, grid_fgg_nc = grid_fxc_nc[0:3]
grid_exc_nc *= grid_weight
grid_fr_nc  *= grid_weight
grid_fg_nc  *= grid_weight
grid_frr_nc *= grid_weight
grid_frg_nc *= grid_weight
grid_fgg_nc *= grid_weight

我们首先生成 PT2 部分的能量．这段代码与 MP2 计算相同，只是需要乘上 PT2 的系数：

In [ ]:
e_ovov = eo[:, None, None, None] - ev[None, :, None, None] + eo[None, None, :, None] - ev[None, None, None, :]
eri0_ovov = eri0_mo[so, sv, so, sv]
E_PT2c = cc_nc * (eri0_ovov * (2 * eri0_ovov - eri0_ovov.swapaxes(1, 3)) / e_ovov).sum()

随后我们只要将刚才的相关矫正加到普通 GGA 的能量就可以了．我们可以与内部版本的 [Gaussian 输出结果](include/basic_gga/xyg3_energy.gjf) 进行比对：

In [ ]:
np.allclose(
    nc_eng.energy_tot(dm=D) + E_PT2c,
    val_from_fchk("Total Energy", "include/basic_gga/xyg3_energy.fchk")
)